In [1]:
import docx
import json

doc = docx.Document('67_2014_QH13_259729.docx')

document_json = {
    "document" : []
}
# Put a place holder for later indexing
for para in doc.paragraphs:
    document_json["document"].append({
        "index" : "0.0.0.0",
        "text" : para.text
	})
    
# save the json file
with open('67_2014_QH13_259729.json', 'w', encoding="utf-8") as json_file:
	json.dump(document_json, json_file, indent=4, ensure_ascii=False)

In [2]:
import docx
import json

doc = docx.Document('86_2015_ND-CP_292146.docx')

document_json = {
    "document" : []
}

for para in doc.paragraphs:
    document_json["document"].append({
        "index" : "0.0.0",
        "text" : para.text
	})
    
# save the json file
with open('86_2015_ND-CP_292146_only_dieu.json', 'w', encoding="utf-8") as json_file:
	json.dump(document_json, json_file, indent=4, ensure_ascii=False)

In [3]:
import re
import json

with open('67_2014_QH13_259729.json', 'r', encoding="utf-8") as json_file:
	data = json.load(json_file)
	document_data = data["document"]

def roman_to_int(s):
	rom_val = {'I': 1, 'V': 5, 'X': 10, 'L': 50, 'C': 100, 'D': 500, 'M': 1000}
	int_val = 0
	for i in range(len(s)):
		if i > 0 and rom_val[s[i]] > rom_val[s[i - 1]]:
			int_val += rom_val[s[i]] - 2 * rom_val[s[i - 1]]
		else:
			int_val += rom_val[s[i]]
	return int_val

document_data_len = len(document_data)

for i in range(1, document_data_len):
	previous_element_index_list = document_data[i-1]["index"].split(".")
	previous_element_chapter_number = int(previous_element_index_list[0])
	previous_element_section_number = int(previous_element_index_list[1])
	previous_element_subsection_number = int(previous_element_index_list[2])
	element = document_data[i]
	# if element["text"] starts with "Chương" then it is a chapter, update the index
	if element["text"].startswith("Chương"):
		# chapter number is the first word right after "Chương"
		chapter_number = element["text"].split()[1]
		# right now the chapter number is roman numeral, convert it to arabic numeral
		chapter_number = roman_to_int(chapter_number)
		# update the index
		index_list = element["index"].split(".")
		index_list[0] = str(chapter_number)
		element["index"] = ".".join(index_list)
	elif element["text"].startswith("Điều"):
		# section number is the first word right after "Điều", remove the dot at the end
		section_number = element["text"].split()[1][:-1]
		# update the index
		index_list = element["index"].split(".")
		index_list[0] = str(previous_element_chapter_number)
		index_list[1] = section_number
		element["index"] = ".".join(index_list)
	elif re.match(r"^\d+\.", element["text"]):
		# subsection number is the first word, remove the dot at the end
		subsection_number = element["text"].split(".")[0]
		# update the index
		index_list = element["index"].split(".")
		index_list[2] = subsection_number
		index_list[1] = str(previous_element_section_number)
		index_list[0] = str(previous_element_chapter_number)
		element["index"] = ".".join(index_list)
	# else if element["text"] starts with a character and a ) then it is a point, update the index
	elif re.match(r"^[a-zđê]\)", element["text"], re.IGNORECASE):
		# point number is the first character, remove the bracket at the end
		point_number = element["text"].split(")")[0]
		# update the index
		index_list = element["index"].split(".")
		index_list[3] = point_number
		index_list[2] = str(previous_element_subsection_number)
		index_list[1] = str(previous_element_section_number)
		index_list[0] = str(previous_element_chapter_number)
		element["index"] = ".".join(index_list)
	else:
		index_list = element["index"].split(".")
		index_list[0] = str(previous_element_chapter_number)
		index_list[1] = str(previous_element_section_number)
		element["index"] = ".".join(index_list)

# save document_data to json file
with open('67_2014_QH13_259729_index.json', 'w', encoding="utf-8") as json_file:
	json.dump(data, json_file, indent=4, ensure_ascii=False)	

In [4]:
import re
import json

with open('86_2015_ND-CP_292146_only_dieu.json', 'r', encoding="utf-8") as json_file:
	data = json.load(json_file)
	document_data = data["document"]

document_data_len = len(document_data)
data_for_print = []

for i in range(1, document_data_len):
	previous_element_index_list = document_data[i-1]["index"].split(".")
	previous_element_section_number = int(previous_element_index_list[0])
	previous_element_subsection_number = int(previous_element_index_list[1])
	element = document_data[i]
	if element["text"].startswith("Chương"):
		# remove that element from the list
		continue
	# elif element["text"] starts with "Mục " and a number
	elif re.match(r"^Mục \d+", element["text"]):
		# remove that element from the list
		continue
	elif element["text"].startswith("Điều"):
		# section number is the first word right after "Điều", remove the dot at the end
		section_number = element["text"].split()[1][:-1]
		# update the index
		index_list = element["index"].split(".")
		index_list[0] = section_number
		element["index"] = ".".join(index_list)
		data_for_print.append({
			"index" : element["index"],
			"text" : element["text"]
		})
	elif re.match(r"^\d+\.", element["text"]):
		# subsection number is the first word, remove the dot at the end
		subsection_number = element["text"].split(".")[0]
		# update the index
		index_list = element["index"].split(".")
		index_list[1] = subsection_number
		index_list[0] = str(previous_element_section_number)
		element["index"] = ".".join(index_list)
		data_for_print.append({
			"index" : element["index"],
			"text" : element["text"]
		})
	# else if element["text"] starts with a character and a ) then it is a point, update the index
	elif re.match(r"^[a-zđê]\)", element["text"], re.IGNORECASE):
		# point number is the first character, remove the bracket at the end
		point_number = element["text"].split(")")[0]
		# update the index
		index_list = element["index"].split(".")
		index_list[2] = point_number
		index_list[1] = str(previous_element_subsection_number)
		index_list[0] = str(previous_element_section_number)
		element["index"] = ".".join(index_list)
		data_for_print.append({
			"index" : element["index"],
			"text" : element["text"]
		})
	else:
		index_list = element["index"].split(".")
		index_list[0] = str(previous_element_section_number)
		element["index"] = ".".join(index_list)
		data_for_print.append({
			"index" : element["index"],
			"text" : element["text"]
		})

# save document_data to json file
with open('86_2015_ND-CP_292146_only_dieu_index.json', 'w', encoding="utf-8") as json_file:
	json.dump({
		"document" : data_for_print
	}, json_file, indent=4, ensure_ascii=False)

In [5]:
import json

with open('86_2015_ND-CP_292146_only_dieu_index.json', 'r', encoding="utf-8") as json_file:
	data = json.load(json_file)
	document_data = data["document"]

document_index = {}
document_data_len = len(document_data)

for i in range(document_data_len):
	element = document_data[i]
	if element["index"] not in document_index:
		document_index.update({element["index"] : element["text"]})
	else:
		document_index[element["index"]] += " " + element["text"]

# save document_index to json file
with open('86_2015_ND-CP_292146_only_dieu_index_another_way.json', 'w', encoding="utf-8") as json_file:
	json.dump(document_index, json_file, indent=4, ensure_ascii=False)